In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

from os import listdir
listdir('drive/MyDrive/nis_2021_json')

import numpy as np
import time

In [ ]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('drive/MyDrive/nis_2021_json/NIS2021-2-6b794392fe59.json', scope)
client = gspread.authorize(creds)

In [ ]:
table_name = 'НИС 1 курс 2021'
#table_name = 'НИС 1 курс 2021 (copy)'

sheet = client.open(table_name).worksheet('Список задач')
sheet_log = client.open(table_name).worksheet('log')


form_name = 'Выбор задачи 2021 (Ответы)'
forma = client.open(form_name).sheet1

# Extract and print all of the values
list_of_hashes = forma.get_all_records()
dfforma = pd.DataFrame(list_of_hashes)
dfforma.set_index('Отметка времени')
#dfforma.head(5)

In [ ]:
list_of_hashes = sheet.get_all_records()
dfsheet = pd.DataFrame(list_of_hashes)

In [ ]:
all_students = dfsheet['Студент'].tolist()
all_students = [x.split(',') for x in all_students if x != '']
lst = []
for x  in all_students:
    lst.extend(x)
    
for i, x in enumerate(lst):
    if x[0] == ' ':
        lst[i] = x[1:]
        
students = set(lst)
len(students)

In [ ]:
ind_last = np.where(dfforma['статус'].to_numpy()=='')[0][0]

In [ ]:
last_update = dfforma['Отметка времени'].iloc[ind_last]
print(last_update)#'06.02.2018 17:06:12'
dfforma = dfforma.loc[dfforma['Отметка времени'] >= last_update]
dfforma.shape[0]

In [ ]:
for index, row in dfforma.iterrows():
    time.sleep(5)
    print(index,row['Фамилия Имя Отчество'])
    if row['Фамилия Имя Отчество'] in students:
        print(row['Фамилия Имя Отчество'], ' already added')
        forma.update_cell(index+2, 12, 'already added')
        continue
    lst_sheet_log = [row.tolist()[0]]+row.tolist()[2:]
    #print(lst_sheet_log)
    status = 'Все указанные темы уже заняты'
    #break
    if row['Тема из предложенного списка или своя?'] == 'Из предложенного списка':
        for n in ['первый', 'второй', 'третий']:
            try:
                print(row['Название выбранной задачи ({0} приоритет)'.format(n)])
                if not row['Название выбранной задачи ({0} приоритет)'.format(n)]:
                    status = 'Пустая тема '+n+' приоритет'
                    forma.update_cell(index+2, 12, status)
                    break
                cell = sheet.find(row['Название выбранной задачи ({0} приоритет)'.format(n)])
            except gspread.exceptions.GSpreadException:
                print('not found')
                status = 'Название темы не найдено'
                forma.update_cell(index+2, 12, status)
                continue
            if not sheet.cell(cell.row, 3).value:
                if row['Проект групповой?'] == 'Нет':
                    sheet.update_cell(cell.row, 3, row['Фамилия Имя Отчество'])
                else:
                    sheet.update_cell(cell.row, 3, row['Фамилия Имя Отчество']+', '+
                              row['Перечислите ФИО других участников проекта (если проект не групповой, то поставьте "-")'])
                    sheet.update_cell(cell.row, 4, row['Группа'])
                    status = 'OK'
                    forma.update_cell(index+2, 12, status)
                    break
    else:
        if row['Проект групповой?'] == 'Нет':
            lst = [row['Название выбранной задачи (первый приоритет)'],
                          row['ФИО руководителя'],
                          row['Фамилия Имя Отчество'],
                          row['Группа']] 
        else: 
            lst = [row['Название выбранной задачи (первый приоритет)'],
                          row['ФИО руководителя'],
                          row['Фамилия Имя Отчество']+', '+
                          row['Перечислите ФИО других участников проекта (если проект не групповой, то поставьте "-")'],
                          row['Группа']] 
        sheet.append_row(lst)
        status = 'OK'
        forma.update_cell(index+2, 12, status)
    if status == 'Все указанные темы уже заняты':
        forma.update_cell(index+2, 12, status)
    lst_sheet_log.append(status)
    #print(lst_sheet_log)
    sheet_log.append_row(lst_sheet_log)

    if status == 'OK' and row['Проект групповой?'] == 'Нет':
        students.add(row['Фамилия Имя Отчество'])
        print('set ',len(students))
    elif status == 'OK' and row['Проект групповой?'] == 'Да':
        students.add(row['Фамилия Имя Отчество'])
        lst = row['Перечислите ФИО других участников проекта (если проект не групповой, то поставьте "-")'].split(',')
        for i, x in enumerate(lst):
            if x[0] == ' ':
                lst[i] = x[1:]
        students.update(lst)
        print('set ',len(students))
        
last_update = row['Отметка времени']
last_update

In [ ]:
len(students)